In [1]:
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import json
import os
import time
import traceback
import random

In [2]:
url = 'https://www.cij.gov.ar/sentencias.html'

In [5]:
#service = Service()
#options = webdriver.ChromeOptions()
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [6]:
wait = WebDriverWait(driver, 10)

tribunal_dropdown_element = wait.until(
    EC.element_to_be_clickable(
        (By.XPATH, '/html/body/main/div[13]/div/div/div[1]/div/div/div[3]/div/div/form[1]/div[2]/div/div[2]/select')
    )
)

tribunal_dropdown_select = Select(tribunal_dropdown_element)
tribunal_dropdown_select.select_by_visible_text(
    "CÁMARA NACIONAL DE APELACIONES EN LO COMERCIAL"
)

In [7]:
fecha_input = wait.until(EC.element_to_be_clickable((By.ID, "fecha_fallo_desde_aux")))
fecha_input.clear()
fecha_input.send_keys("01/01/2024")
driver.execute_script(
    "arguments[0].dispatchEvent(new Event('change'));",
    fecha_input
)

In [9]:
buscar_btn = wait.until(EC.element_to_be_clickable(
    (By.XPATH, '/html/body/main/div[13]/div/div/div[1]/div/div/div[3]/div/div/form[1]/div[6]/input[1]')
))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", buscar_btn)
buscar_btn.click()

In [27]:
JSON_PATH = "fallos.json"

# --- 1) Intento cargar progresos anteriores ---
fallos_all = []
current_page = 1
try:
    with open(JSON_PATH, "r", encoding="utf-8") as f:
        fallos_all = json.load(f)
        # deduzco la página en la que me quedé,
        # asumiendo 20 resultados por página:
        current_page = len(fallos_all) // 20 + 1
        print(f"Reanudando desde página {current_page}")
except FileNotFoundError:
    print("Empezando desde cero")

Reanudando desde página 381


In [10]:
def scrape_pagina():
    """Lee los <div.result> y devuelve sus datos como lista de dicts."""
    # ya no devolvemos elementos, solo datos
    datos_pagina = []
    elems = driver.find_elements(By.CSS_SELECTOR, "div.result")
    for el in elems:
        d = {}
        for li in el.find_elements(By.CSS_SELECTOR, "ul.info li"):
            etiqueta = li.find_element(By.CSS_SELECTOR, "span.s2").text.rstrip(":")
            valor = li.text.replace(li.find_element(By.CSS_SELECTOR, "span.s2").text, "").strip()
            d[etiqueta] = valor
        d["DownloadURL"] = el.find_element(By.CSS_SELECTOR, "a.download").get_attribute("href")
        datos_pagina.append(d)
    return elems, datos_pagina

In [11]:
try:
    while True:
        print(f"Scrapeando página {current_page}…")

        # 1) Aseguramos que los resultados estén cargados
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.result")))

        # 2) Capturamos both WebElements y datos
        elementos, nuevos = scrape_pagina()
        fallos_all.extend(nuevos)

        # 3) Guardamos el JSON incrementalmente
        with open(JSON_PATH, "w", encoding="utf-8") as f:
            json.dump(fallos_all, f, ensure_ascii=False, indent=2)
        print(f"  → {len(nuevos)} registros añadidos (total: {len(fallos_all)})")

        # 4) Click en “Siguiente”
        siguiente = wait.until(EC.element_to_be_clickable((
            By.XPATH,
            '/html/body/main/div[13]/div/div/div[1]/div/div/div[5]/div[2]/a[9]'
        )))
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", siguiente)
        siguiente.click()

        # 5) Esperamos a que el primer elemento anterior desaparezca
        wait.until(EC.staleness_of(elementos[0]))
        current_page += 1
        time.sleep(0.5)

except Exception as e:
    print(f"\n⚠️  Error en la página {current_page}: {e}")
    traceback.print_exc()

Scrapeando página 288…
  → 20 registros añadidos (total: 5760)
Scrapeando página 289…
  → 20 registros añadidos (total: 5780)
Scrapeando página 290…
  → 20 registros añadidos (total: 5800)
Scrapeando página 291…
  → 20 registros añadidos (total: 5820)
Scrapeando página 292…
  → 20 registros añadidos (total: 5840)
Scrapeando página 293…
  → 20 registros añadidos (total: 5860)
Scrapeando página 294…
  → 20 registros añadidos (total: 5880)
Scrapeando página 295…
  → 20 registros añadidos (total: 5900)
Scrapeando página 296…
  → 20 registros añadidos (total: 5920)
Scrapeando página 297…
  → 20 registros añadidos (total: 5940)
Scrapeando página 298…
  → 20 registros añadidos (total: 5960)
Scrapeando página 299…
  → 20 registros añadidos (total: 5980)
Scrapeando página 300…
  → 20 registros añadidos (total: 6000)
Scrapeando página 301…
  → 20 registros añadidos (total: 6020)
Scrapeando página 302…
  → 20 registros añadidos (total: 6040)
Scrapeando página 303…
  → 20 registros añadidos (total

KeyboardInterrupt: 

In [28]:
JSON_PATH = "fallos.json"

# --- 1) Intento cargar progresos anteriores ---
fallos_all = []
current_page = 1
try:
    with open(JSON_PATH, "r", encoding="utf-8") as f:
        fallos_all = json.load(f)
        # deduzco la página en la que me quedé,
        # asumiendo 20 resultados por página:
        current_page = len(fallos_all) // 20 + 1
        print(f"Reanudando desde página {current_page}")
except FileNotFoundError:
    print("Empezando desde cero")

Reanudando desde página 381


In [30]:
# Leer el archivo JSON y convertirlo a un DataFrame
fallos_df = pd.read_json(JSON_PATH, encoding="utf-8").drop_duplicates().reset_index(drop=True)

# Mostrar las primeras filas del DataFrame
fallos_df['Sala'] = fallos_df['Tribunal'].apply(lambda x: x.split(' - ')[1])
display(fallos_df.info())
display(fallos_df.head())
fallos_df.to_csv('fallos.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339 entries, 0 to 4338
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Tribunal            4339 non-null   object
 1   Expediente N°       4339 non-null   object
 2   Carátula            4339 non-null   object
 3   Fecha de sentencia  4339 non-null   object
 4   DownloadURL         4339 non-null   object
 5   Sala                4339 non-null   object
dtypes: object(6)
memory usage: 203.5+ KB


None

,Tribunal,Expediente N°,Carátula,Fecha de sentencia,DownloadURL,Sala
0,CAMARA COMERCIAL - SALA F,COM 001110/2023/CA001,"TORRES, NOELI AYELEN c/ CALEDONIA COMPAÑIA ARG...",05/05/2025,https://www.cij.gov.ar/d/sentencia-SGU-b99ac9c...,SALA F
1,CAMARA COMERCIAL - SALA A,COM 006633/2025/CA001,SUPERINTENDENCIA DE RIESGOS DEL TRABAJO c/ LA ...,05/05/2025,https://www.cij.gov.ar/d/sentencia-SGU-2259385...,SALA A
2,CAMARA COMERCIAL - SALA F,COM 006042/2023/CA003,"FERRARI, ROBERTO EUGENIO Y OTRO c/ AMERICAN AI...",05/05/2025,https://www.cij.gov.ar/d/sentencia-SGU-3295990...,SALA F
3,CAMARA COMERCIAL - SALA F,CIV 058301/2023/CA001,"CARNICER, HECTOR MARIANO c/ DAVID, LORENA Y OT...",05/05/2025,https://www.cij.gov.ar/d/sentencia-SGU-8af2a80...,SALA F
4,CAMARA COMERCIAL - SALA F,COM 021297/2024/CA001,"TORRES LUYO, DIEGO LUIS Y OTRO c/ SCLAVUNO, AN...",05/05/2025,https://www.cij.gov.ar/d/sentencia-SGU-594abfc...,SALA F
